### Tbot - Trade Bot

 Tbot is a Robot of trade developed to trade cryptourrencies or s&p500 stocks, based on https/json queries. With few adjustments it may help traders to improve their workflow. In this file we shall compile three steps followed at the "Algorithimic Trading Using Python" course created by the api developer and completed by the developer of this api. ("https://freecodecamp.org")
 
### Library Imports
The first thing we need to do is import the open-source software libraries that we'll be using in this api. 

In [1]:
import numpy as np
import pandas as pd
import requests
import secrets
import xlsxwriter
import math
import hmac
import time
import datetime
import hashlib
import binascii

### API token, secret key and acess keys
API tokens (and other sensitive information) shall be stored in a secrets.py file that doesn't get pushed to your local Git repository. The AcessKey and SecretKey are provided by the server admin.

In [2]:
from secrets import SecretKey
from secrets import AccessKey
from novadax import RequestClient as NovaClient

nova_client = NovaClient(AccessKey, SecretKey)

### Importing tht List of currencies, making the API Call and the table Dataframe
List of crypto-currencies, get balance, and table creation

In [3]:
balance = nova_client.get_account_balance()
bdata = balance['data']

bcolumns = ['Currency', 'Balance']
bdataframe = pd.DataFrame(columns = bcolumns,)
bs = 0
bcr = {}

for bsymbol in bdata:
    pdis = bdata[bs]
    bis = pdis['balance']
    bic = pdis['currency']
    bcr[f"{bic}"] = f"{bis}"   
    bdataframe.append(
        pd.Series([
            bic,
            bis,
        ], index = bcolumns),
        ignore_index = True)
    bdataframe = bdataframe.append(
    pd.Series([
        bic,
        bis,
    ], index=bcolumns),
        ignore_index=True)
    bs = bs + 1  

bdataframe.sort_values('Balance', ascending = False, inplace = True)
bdataframe.reset_index(drop = True, inplace = True)
bdataframe

,Currency,Balance
0,BTT,96.09424
1,USDT,0.09682579
2,DGB,0.0353
3,LINK,0.0088601
4,DOGE,0.00881
5,XLM,0.00447
6,NULS,0.0044
7,IOTA,0.003905
8,XRP,0.0036
9,BTC,0.00041766


### Dolar price, timestamp and symbols 
Important cell defining symbols and the formula to the dolar price in terms of brazilian real(BRL), wich can be easily translated to other currencies or symbols.

In [4]:
symbol = 'ETH_USDT'
symbols = ['ETH_USDT']
currency = 'R$'
dsymbol = 'ETH_BRL'
ldolar = nova_client.list_trades(symbol, limit = 5)
my_columns = ['Symbol', 'Amount', 'Price', 'Number of Shares to Buy']
ldr_dataframe = pd.DataFrame(columns = my_columns)

for symbol in symbols:
    f = 0
    for i in ldolar['data']:
        ldolar = nova_client.list_trades(symbol, limit=10)
        ldar = ldolar['data']
        ldr = ldar[f]
        amount = ldr['amount']
        price = float(ldr['price'])
        cprice = float(amount) * price
        edolar = bcr['USDT']
        eth_wal = bcr['ETH']
        
        cwal = float(eth_wal) * price + float(edolar)
        
        sharestobuy = float(cwal/price)
                            
        
        baseVolume24h =  'N/A'
        tsp = (ldr['timestamp'])
        f = f + 1
        ldr_dataframe.append(
                    pd.Series(
                    [
                        symbol,
                        amount,
                        price,
                        sharestobuy,
                    ]
                    ,index = my_columns),
                ignore_index=True)

    
        ldr_dataframe = ldr_dataframe.append(
                pd.Series([
                        symbol,
                        amount,
                        price,
                        sharestobuy,
                    
                      ]
                ,index = my_columns), 
                ignore_index = True)

ldr_dataframe.sort_values('Price', ascending = True, inplace = True)
ldr_dataframe.reset_index(drop = True, inplace = True)
ldr_dataframe

,Symbol,Amount,Price,Number of Shares to Buy
0,ETH_USDT,0.0572,1532.32,0.000063
1,ETH_USDT,0.0119,1540.00,0.000063
2,ETH_USDT,0.2781,1542.14,0.000063
3,ETH_USDT,0.0205,1544.54,0.000063
4,ETH_USDT,0.0781,1545.15,0.000063


### Symbol position
For the execution to be done independently for each symbol, the below cell is needed:

In [5]:
my_columns = ['Symbol', 'Currency Price', 'Number of Shares to Buy']
e_dataframe = pd.DataFrame(columns = my_columns)
for symbol in symbols:
    r = 0
    for i in ldolar['data']:  
        fddata = nova_client.list_trades(symbol, limit=10)
        fdata = fddata['data']
        fdt = fdata[r]
        price = fdt['price']
        amount = fdt['amount']
        cprice = float(amount) * float(price)
        edolar = bcr['USDT']
        eth_wal = float(bcr['ETH']) * float(price)
        edolarwal = float(bcr['USDT']) + float(eth_wal)
        shares_to_buy = (float(edolarwal)/float(price))
        etsp = (fdt['timestamp'])
        r = r + 1
        for i in fddata['data'][:1]:
            e_dataframe.append(
                pd.Series(
                    [
                        symbol,
                        price,
                        shares_to_buy
                    ]
                    ,index = my_columns),
                ignore_index=True)
    
    
            e_dataframe = e_dataframe.append(
                                                pd.Series([
                                                    symbol, 
                                                    price,
                                                    shares_to_buy
                                                    ],
                                                    index = my_columns
                                                    ),
                                                ignore_index = True)

e_dataframe

,Symbol,Currency Price,Number of Shares to Buy
0,ETH_USDT,1545.15,0.000063
1,ETH_USDT,1544.54,0.000063
2,ETH_USDT,1542.14,0.000063
3,ETH_USDT,1540,0.000063
4,ETH_USDT,1532.32,0.000063
5,ETH_USDT,1539.74,0.000063
6,ETH_USDT,1469.55,0.000066
7,ETH_USDT,1497.51,0.000065
8,ETH_USDT,1482.7,0.000065
9,ETH_USDT,1492.93,0.000065


### Create the momentum table
For mommentum we use the last order minus the order from an amount x of time. The place from wich the data comes is main importance on the formula. As stated in the comment section on the first versions, data must come from the common filled orders or the last trades from the users history, here the second case is uded, as well from this version on, but the first ones used common filled orders.

In [23]:
#20210222#v0-9#import all the list itens from the last dataframes and send them to the xlsx file. 
#20200225#v0-9-1# the data must come from the common filled orders or the last trades from the users history, 
#                 here the second case is uded, as well from this version on, first ones used common filled orders
#orders
listtrades = nova_client.list_orders(symbol, 'FILLED', limit=5)
ldata = listtrades['data']
lindice= ldata[0]
amount = lindice['amount']
my_columns = ['Symbol', 'Amount', 'Number of Shares to Buy','Last Orders Price', 'B/S', 'Momentum Price',  'momentum-raw-data']
m_dataframe = pd.DataFrame(columns = my_columns)

for symbol in symbols:
    listtrades = nova_client.list_orders(symbol, 'FINISHED')
    s = 0
    for i in listtrades['data'][:9]:
        ldata = listtrades['data']
        ldc = ldata[s]
        amount = ldc['amount']
        price = float(ldc['price'])
        
        amount = ldc['amount']
        cprice = float(amount) * float(price)
        edolar = bcr['USDT']
        eth_wal = float(bcr['ETH']) * float(price)
        edolarwal = float(bcr['USDT']) + float(eth_wal)
#ref:        https://stackoverflow.com/questions/41383787/round-down-to-2-decimal-in-python
        sharestobuy = float(amount) - (float(edolarwal))/float(price)
        bs = ldc['side']
        order = float(amount) - float(sharestobuy)
        srto = float(order)
        
        #M = V - Vx ; M representa o indicador de momentum, V o valor de fechamento atual e V(x_diasatras) o valor anterior
        #nao é deste modo que se utiliza o simbolo x na multiplicação. Vx seriam multiplos valores de V em x dias. Para este caso, Vi
        #veja que por conotaçao convencional parece nao fazer diferença, porém, na matemática, tais convenções são necessárias e de certo modo indispensáveis no tempo t
        #h = 
        vxdata = e_dataframe['Currency Price']
        vxi = vxdata[s]
        vx = vxi
        momentum = float(price) - float(vx)
        tsp = (ldc['timestamp'])
        s = s + 1
        m_dataframe.append(
                    pd.Series(
                    [
                        symbol,
                        amount,
                        sharestobuy,
                        price,
                        bs,
                        vx,
                        momentum,
                    ]
                    ,index = my_columns),
                ignore_index=True)

    
        m_dataframe = m_dataframe.append(
                pd.Series([
                        symbol,
                        amount,
                        sharestobuy,
                        price,
                        bs,
                        vx,
                        momentum,
                    
                      ]
                ,index = my_columns), 
                ignore_index = True)

m_dataframe.sort_values('Last Orders Price', ascending = True, inplace = True)
m_dataframe.reset_index(drop = True, inplace = True)
m_dataframe

,Symbol,Amount,Number of Shares to Buy,Last Orders Price,B/S,Momentum Price,momentum-raw-data
0,ETH_USDT,0.0188,0.018750,1928.42,BUY,1539.74,388.68
1,ETH_USDT,0.0685,0.068450,1953.10,BUY,1540,413.10
2,ETH_USDT,0.05,0.049950,1953.10,BUY,1532.32,420.78
3,ETH_USDT,0.0186,0.018550,1953.26,SELL,1497.51,455.75
4,ETH_USDT,0.0186,0.018550,1953.42,SELL,1469.55,483.87
5,ETH_USDT,0.0186,0.018551,1967.68,SELL,1482.7,484.98
6,ETH_USDT,0.0675,0.067451,1983.00,BUY,1542.14,440.86
7,ETH_USDT,0.0674,0.067352,1999.85,SELL,1545.15,454.70
8,ETH_USDT,0.0674,0.067352,2034.00,SELL,1544.54,489.46


### Sending the requisition to the web
The content in the order requisition should be sent to the server following the instructions given by the company or individual that holds the coin. Here as columns the last used titles shall be all used in one go to give a more complete dataframe in our xlsx database.

In [27]:
#sort price [ok]
#wallet_amount [ok]
#dataframe momentum[ok]pt2[ok]
#buy lower(wallet_amount - amount)     
#sell + profit(constant*market momentum)
#if sell = true; [buy lower < sell][(sell - (contant*momentum)/2 or (buy + constant/momentum)/2
#P = S - B; SUM(Pi to Pn)  = Pi{(S - D) or (S - B)}
#in the if statement one more contidional is needed at the "or" statment. If or else(momentum>=x or else)
#20210222#v0-9#import all the list itens from the last dataframes and send them to the xlsx file. 
#20200225#v0-9-1# the data must come from the common filled orders or the last trades from the users history, 
#                 here the second case is uded, as well from this version on, first ones used common filled orders

4.921279786143998e-05


In [68]:
#orders
listtrades = nova_client.list_orders(symbol, 'FILLED', limit=5)
ldata = listtrades['data']
lindice= ldata[0]
amount = lindice['amount']
my_columns = ['Symbol', 'Amount', 'Last Orders Price', 'B/S', 'Momentum Price',  'momentum-raw-data', 'tsp']
m_dataframe = pd.DataFrame(columns = my_columns)

for symbol in symbols:
    listtrades = nova_client.list_orders(symbol, 'FINISHED')
    s = 0
    for i in listtrades['data'][:9]:
        ldata = listtrades['data']
        ldc = ldata[s]
        amount = ldc['amount']
        price = float(ldc['price'])
        
        amount = float(ldc['amount'])
        cprice = float(amount) * float(price)
        edolar = bcr['USDT']
        eth_wal = float(bcr['ETH']) * float(price)
        edolarwal = float(bcr['USDT']) + float(eth_wal)
#ref:        https://stackoverflow.com/questions/41383787/round-down-to-2-decimal-in-python
        bs = ldc['side']
        
        #M = V - Vx ; M representa o indicador de momentum, V o valor de fechamento atual e V(x_diasatras) o valor anterior
        #nao é deste modo que se utiliza o simbolo x na multiplicação. Vx seriam multiplos valores de V em x dias. Para este caso, Vi
        #veja que por conotaçao convencional parece nao fazer diferença, porém, na matemática, tais convenções são necessárias e de certo modo indispensáveis no tempo t
        vxdata = e_dataframe['Currency Price']
        vxi = vxdata[s]
        momentumprice = float(vxi)
        momentumraw = float(price) - float(vx)
        bmprice = 
        tsp = (ldc['timestamp'])
        s = s + 1
        m_dataframe.append(
                    pd.Series(
                    [
                        symbol,
                        amount,
                        price,
                        bs,
                        momentumprice,
                        momentumraw,
                        tsp,
                    ]
                    ,index = my_columns),
                ignore_index=True)

    
        m_dataframe = m_dataframe.append(
                pd.Series([
                        symbol,
                        amount,
                        price,
                        bs,
                        momentumprice,
                        momentumraw,
                        tsp,
                    
                      ]
                ,index = my_columns), 
                ignore_index = True)

m_dataframe.sort_values('tsp', ascending = False, inplace = True)
m_dataframe.reset_index(drop = True, inplace = True)
m_dataframe

,Symbol,Amount,Last Orders Price,B/S,Momentum Price,momentum-raw-data,tsp
0,ETH_USDT,0.0674,1999.85,SELL,1545.15,517.15,1613926598759
1,ETH_USDT,0.0674,2034.00,SELL,1544.54,551.30,1613848118979
2,ETH_USDT,0.0675,1983.00,BUY,1542.14,500.30,1613829098317
3,ETH_USDT,0.0685,1953.10,BUY,1540.00,470.40,1613770692466
4,ETH_USDT,0.0500,1953.10,BUY,1532.32,470.40,1613770657137
5,ETH_USDT,0.0188,1928.42,BUY,1539.74,445.72,1613758520796
6,ETH_USDT,0.0186,1953.42,SELL,1469.55,470.72,1613758491526
7,ETH_USDT,0.0186,1953.26,SELL,1497.51,470.56,1613758459823
8,ETH_USDT,0.0186,1967.68,SELL,1482.70,484.98,1613758415661


In [57]:
#buy lower(wallet_amount - amount)     
#sell + profit(constant*market momentum)
#if sell = true; [buy lower < sell][(sell - (contant*momentum)/2 or (buy + constant/momentum)/2
#P = S - B; SUM(Pi to Pn)  = Pi{(S - D) or (S - B)}
#in the if statement one more contidional is needed at the "or" statment. If or else(momentum>=x or else)
#20210222#v0-9#import all the list itens from the last dataframes and send them to the xlsx file. 
#20200225#v0-9-1# the data must come from the common filled orders or the last trades from the users history, 
#                 here the second case is uded, as well from this version on, first ones used common filled orders


if momentumprice > price :
    sellprofit = math.floor(price + price/100*1/2)/1.00
    result = nova_client.create_order(symbol, 'LIMIT', 'SELL', price = profit, amount = amount)  

if momentumprice < price and amount < 0.02:
    print("Market is not ready for the sell")

if momentumprice < price and amount > 0.02:
    result = nova_client.create_order(symbol, 'LIMIT', 'BUY', price = bmprice, amount = amount)
    

Market is not ready for the sell


### Format and export xlsx file
In the last section of the tbot a xlsx file is saved with the order table for later reading purposes.

In [ ]:
writer = pd.ExcelWriter('orders.xlsx', engine = 'xlsxwriter')
m_dataframe.to_excel(writer, sheet_name = 'orderlist', index = False)

In [ ]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

float_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [ ]:
column_formats = {
    'A':['Symbol', string_template],
    'B':['Amount', float_template],
    'C':['Price', dollar_template],
}

for column in column_formats.keys():
    writer.sheets['orderlist'].set_column(f'{column}:{column}', 25, column_formats[column][1])
    writer.sheets['orderlist'].write(f'{column}1', column_formats[column][0], column_formats[column][1])
   

In [ ]:
writer.save()